# TFX Evaluation

In [ ]:
from importlib import reload
from pathlib import Path
from typing import cast

import tensorflow_model_analysis as tfma
from absl import logging
from google.cloud.storage import Client as GCSClient
from google.cloud.storage.blob import Blob
from tfx import v1 as tfx
from tfx.components import CsvExampleGen, Evaluator, SchemaGen, StatisticsGen
from tfx.dsl.components.common.importer import IMPORT_RESULT_KEY, Importer
from tfx.orchestration.experimental.interactive.interactive_context import (
    InteractiveContext,
)
from tfx.types.standard_artifacts import Model
from tfx.types.standard_component_specs import (
    EVALUATION_KEY,
    EXAMPLES_KEY,
    SCHEMA_KEY,
    STATISTICS_KEY,
)

from recommender_systems import evaluator_module
from recommender_systems.features import ProductFeatures
from recommender_systems.splits import Splits

logging.set_verbosity(logging.INFO)

DATA = Path.cwd().parent / "data"

PIPELINE_NAME = "evaluate"

context = InteractiveContext(
    pipeline_name=PIPELINE_NAME,
    pipeline_root=str(Path("pipeline-root") / PIPELINE_NAME),
)

## Ingest Reviews

### Examples

In [ ]:
reviews_example_gen_component = CsvExampleGen(
    input_base=str(DATA / "reviews"),
    input_config=tfx.proto.Input(
        splits=[tfx.proto.Input.Split(name=Splits.TEST, pattern="test.csv")]
    ),
    output_config=tfx.proto.Output(
        split_config=tfx.proto.SplitConfig(
            splits=[tfx.proto.SplitConfig.Split(name=Splits.TEST, hash_buckets=1)]
        )
    ),
)
context.run(reviews_example_gen_component, enable_cache=True)

### Statistics

In [ ]:
reviews_statistics_gen_component = StatisticsGen(
    examples=reviews_example_gen_component.outputs[EXAMPLES_KEY]
)
context.run(reviews_statistics_gen_component, enable_cache=True)

In [ ]:
context.show(reviews_statistics_gen_component.outputs[STATISTICS_KEY])

### Schema

In [ ]:
reviews_schema_gen_component = SchemaGen(
    statistics=reviews_statistics_gen_component.outputs[STATISTICS_KEY]
)
context.run(reviews_schema_gen_component, enable_cache=True)

In [ ]:
context.show(reviews_schema_gen_component.outputs[SCHEMA_KEY])

## Evaluate Model

In [2]:
PARTICIPANT = "stefan-dominicus"

In [ ]:
blobs: list[Blob] = sorted(
    GCSClient(project="tal-deep-learning-indabax").list_blobs(
        bucket_or_name="tal-deep-learning-indabax-models",
        prefix=f"{PARTICIPANT}/recommender_systems/",
        match_glob="**/saved_model.pb",
    ),
    key=lambda b: b.generation,
)
assert blobs, "No models found in GCS bucket."
source_uri = cast(str, blobs[-1].path).removesuffix("/saved_model.pb")

model_importer_component = Importer(
    source_uri=source_uri, artifact_type=Model, reimport=True
)
context.run(model_importer_component, enable_cache=True)

In [ ]:
reload(evaluator_module)

evaluator_component = Evaluator(
    examples=reviews_example_gen_component.outputs[EXAMPLES_KEY],
    model=model_importer_component.outputs[IMPORT_RESULT_KEY],
    example_splits=[Splits.TEST],
    eval_config=tfma.EvalConfig(
        metrics_specs=[
            tfma.MetricsSpec(
                metrics=[
                    tfma.MetricConfig(
                        class_name="TopKAccuracy",
                        module=evaluator_module.__name__,
                    ),
                ],
            ),
        ],
        model_specs=[
            tfma.ModelSpec(
                label_key=ProductFeatures.ID,
                signature_name="evaluate_products_for_customer",
            ),
        ],
    ),
    schema=reviews_schema_gen_component.outputs[SCHEMA_KEY],
    module_file=evaluator_module.__file__,
)
context.run(evaluator_component, enable_cache=False)

In [ ]:
output_path = evaluator_component.outputs[EVALUATION_KEY].get()[0].uri

# Load the evaluation result
eval_result = tfma.load_eval_result(output_path)
print("EvalResult:", eval_result)
print("\nMetrics:", eval_result.get_metrics_for_slice())

All done!